### Image Blur

In [9]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

# Load the image
image_path = "dog.jpeg"
image = Image.open(image_path)

# Convert the image to grayscale
grayscale_transform = transforms.Grayscale()
grayscale_image = grayscale_transform(image)

# Convert the grayscale image to a PyTorch tensor
transform = transforms.ToTensor()
image_tensor = transform(grayscale_image).unsqueeze(0)

# Define the blur multiplier
blur_multiplier = 1.1

# Define a 5x5 Gaussian blur kernel
gaussian_kernel = torch.tensor([[1., 4., 6., 4., 1.],
                                [4., 16., 24., 16., 4.], 
                                [6., 24., 36., 24., 6.],
                                [4., 16., 24., 16., 4.],
                                [1., 4., 6., 4., 1.]]) / 256

# Multiply the Gaussian kernel by the blur multiplier
gaussian_kernel *= blur_multiplier

# Add batch dimension and channel dimension to the kernel
gaussian_kernel = gaussian_kernel.unsqueeze(0).unsqueeze(0)

# Apply the Gaussian blur using conv2d
output_tensor = F.conv2d(image_tensor, gaussian_kernel, padding=2)

# Convert the output tensor back to a PIL image
output_image = transforms.ToPILImage()(output_tensor.squeeze())

# Save the blurred image
output_image.save("blurred_dog_grayscale.jpeg")


## Blur for RGB all three channels

In [11]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

# Load the image
image_path = "dog.jpeg"
image = Image.open(image_path)

# Convert the image to a PyTorch tensor
transform = transforms.ToTensor()
image_tensor = transform(image).unsqueeze(0)

# Define the blur multipliers for each channel (R, G, B)
blur_multipliers = [1, 1, 1]

# Define a 5x5 Gaussian blur kernel
gaussian_kernel = torch.tensor([[1., 4., 6., 4., 1.],
                                [4., 16., 24., 16., 4.], 
                                [6., 24., 36., 24., 6.],
                                [4., 16., 24., 16., 4.],
                                [1., 4., 6., 4., 1.]]) / 256

# Create an empty tensor to store the blurred image
blurred_image_tensor = torch.zeros_like(image_tensor)

# Apply Gaussian blur to each channel separately
for c in range(3):
    # Multiply the Gaussian kernel by the blur multiplier for the current channel
    channel_kernel = gaussian_kernel * blur_multipliers[c]
    
    # Add batch dimension and channel dimension to the kernel
    channel_kernel = channel_kernel.unsqueeze(0).unsqueeze(0)
    
    # Apply the Gaussian blur to the current channel using conv2d
    blurred_image_tensor[:, c, :, :] = F.conv2d(image_tensor[:, c, :, :].unsqueeze(1), channel_kernel, padding=2).squeeze(1)

# Convert the blurred image tensor back to a PIL image
blurred_image = transforms.ToPILImage()(blurred_image_tensor.squeeze())

# Save the blurred image
blurred_image.save("blurred_dog_rgb.jpeg")


## Edge Detection

In [13]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

# Load the image
image_path = "dog.jpeg"
image = Image.open(image_path)

# Convert the image to grayscale
grayscale_transform = transforms.Grayscale()
grayscale_image = grayscale_transform(image)

# Convert the grayscale image to a PyTorch tensor
transform = transforms.ToTensor()
image_tensor = transform(grayscale_image).unsqueeze(0)

# Define the Sobel kernels for edge detection
sobel_x = torch.tensor([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)

sobel_y = torch.tensor([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)

# Apply the Sobel kernels to the image using convolution
edges_x = F.conv2d(image_tensor, sobel_x, padding=1)
edges_y = F.conv2d(image_tensor, sobel_y, padding=1)

# Compute the magnitude of the edges
edges = torch.sqrt(edges_x**2 + edges_y**2)

# Normalize the edge values to [0, 1]
edges = (edges - edges.min()) / (edges.max() - edges.min())

# Convert the edge tensor back to a PIL image
edges_image = transforms.ToPILImage()(edges.squeeze())

# Save the edges image
edges_image.save("dog_edges.jpeg")
